In [ ]:
import os,json
data_root="C:/Users/HP/Downloads/Emotion-Analysis/dataset"
def load_json(filename):
    """Load JSON file and return data."""
    with open(filename, "r") as f:
        return json.load(f)
train_data = load_json(f"{data_root}/train.json")
val_data = load_json(f"{data_root}/val.json")
test_data = load_json(f"{data_root}/test.json")
print("Sample train entry:", train_data[0]) 
print("Sample val entry:", val_data[0])
print("Sample test entry:", test_data[0])
def fix_path(path):
    return os.path.normpath(os.path.join(data_root, path)).replace("\\", "/")
train_images = [(entry[0], fix_path(entry[1])) for entry in train_data]
val_images = [(entry[0], fix_path(entry[1])) for entry in val_data]
test_images = [(entry[0], fix_path(entry[1])) for entry in test_data]
print(f"Total train images: {len(train_images)}")
print(f"Total validation images: {len(val_images)}")
print(f"Total test images: {len(test_images)}")

Sample train entry: ['amusement', 'image/amusement/amusement_08258.jpg', 'annotation/amusement/amusement_08258.json']
Sample val entry: ['amusement', 'image/amusement/amusement_03317.jpg', 'annotation/amusement/amusement_03317.json']
Sample test entry: ['awe', 'image/awe/awe_06628.jpg', 'annotation/awe/awe_06628.json']
Total train images: 94481
Total validation images: 5905
Total test images: 17716


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import json
from PIL import Image
class EmoSet(Dataset):
    ATTRIBUTES_MULTI_CLASS = [
        'scene', 'facial_expression', 'human_action', 'brightness', 'colorfulness',
    ]
    ATTRIBUTES_MULTI_LABEL = [
        'object'
    ]
    NUM_CLASSES = {
        'brightness': 11,
        'colorfulness': 11,
        'scene': 254,
        'object': 409,
        'facial_expression': 6,
        'human_action': 264,
    }
    def __init__(self,
                 data_root,
                 num_emotion_classes,
                 phase,
                 ):
        assert num_emotion_classes in (8, 2)
        assert phase in ('train', 'val', 'test')
        self.transforms_dict = self.get_data_transforms()
        self.info = self.get_info(data_root, num_emotion_classes)
        if phase == 'train':
            self.transform = self.transforms_dict['train']
        elif phase == 'val':
            self.transform = self.transforms_dict['val']
        elif phase == 'test':
            self.transform = self.transforms_dict['test']
        else:
            raise NotImplementedError
        data_store = json.load(open(os.path.join(data_root, f'{phase}.json')))
        self.data_store = [
            [
                self.info['emotion']['label2idx'][item[0]],
                os.path.join(data_root, item[1]),
                os.path.join(data_root, item[2])
            ]
            for item in data_store
        ]
    @classmethod
    def get_data_transforms(cls):
        transforms_dict = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }
        return transforms_dict
    def get_info(self, data_root, num_emotion_classes):
        assert num_emotion_classes in (8, 2)
        info = json.load(open(os.path.join(data_root, 'info.json')))
        if num_emotion_classes == 8:
            pass
        elif num_emotion_classes == 2:
            emotion_info = {
                'label2idx': {
                    'amusement': 0,
                    'awe': 0,
                    'contentment': 0,
                    'excitement': 0,
                    'anger': 1,
                    'disgust': 1,
                    'fear': 1,
                    'sadness': 1,
                },
                'idx2label': {
                    '0': 'positive',
                    '1': 'negative',
                }
            }
            info['emotion'] = emotion_info
        else:
            raise NotImplementedError
        return info
    def load_image_by_path(self, path):
        image = Image.open(path).convert('RGB')
        image = self.transform(image)
        return image
    def load_annotation_by_path(self, path):
        json_data = json.load(open(path))
        return json_data
    def __getitem__(self, item):
        emotion_label_idx, image_path, annotation_path = self.data_store[item]
        image = self.load_image_by_path(image_path)
        annotation_data = self.load_annotation_by_path(annotation_path)
        data = { 'image': image, 'emotion_label_idx': emotion_label_idx}
        for attribute in self.ATTRIBUTES_MULTI_CLASS:
            attribute_label_idx = -1
            if attribute in annotation_data:
                attribute_label_idx = self.info[attribute]['label2idx'][str(annotation_data[attribute])]
            data.update({f'{attribute}_label_idx': attribute_label_idx})
        for attribute in self.ATTRIBUTES_MULTI_LABEL:
            assert attribute == 'object'
            num_classes = self.NUM_CLASSES[attribute]
            attribute_label_idx = torch.zeros(num_classes)
            if attribute in annotation_data:
                for label in annotation_data[attribute]:
                    attribute_label_idx[self.info[attribute]['label2idx'][label]] = 1
            data.update({f'{attribute}_label_idx': attribute_label_idx})
        return data
    def __len__(self):
        return len(self.data_store)
if __name__ == '__main__':
    data_root = "C:/Users/HP/Downloads/Emotion-Analysis/dataset"
    num_emotion_classes = 8
    phase = 'train'
    dataset = EmoSet(
        data_root=data_root,
        num_emotion_classes=num_emotion_classes,
        phase=phase,
    )
    dataloader = DataLoader(dataset, batch_size = 256, shuffle = True)
    print(dataloader)
    for i,data in enumerate(dataloader):
        print(data['emotion_label_idx'])
        print(data['scene_label_idx'])
        print(data['facial_expression_label_idx'])
        print(data['human_action_label_idx'])
        print(data['brightness_label_idx'])
        print(data['colorfulness_label_idx'])
        print(data['object_label_idx'])
        break

tensor([0, 7, 3, 1, 7, 7, 7, 5, 0, 1, 2, 2, 3, 0, 7, 3, 1, 3, 7, 3, 4, 4, 7, 1,
        1, 2, 6, 4, 5, 3, 0, 2, 3, 2, 7, 0, 3, 1, 4, 3, 4, 6, 5, 5, 6, 4, 1, 2,
        4, 2, 6, 3, 0, 7, 0, 3, 2, 3, 6, 2, 1, 7, 1, 0, 3, 3, 0, 1, 5, 7, 1, 2,
        0, 1, 4, 5, 2, 2, 0, 1, 2, 1, 1, 2, 0, 1, 6, 7, 5, 1, 1, 5, 7, 3, 0, 2,
        7, 6, 2, 1, 6, 4, 3, 0, 7, 1, 3, 3, 7, 0, 1, 6, 5, 2, 2, 1, 1, 7, 7, 0,
        6, 3, 2, 4, 4, 5, 3, 1, 1, 6, 1, 6, 1, 2, 0, 2, 3, 3, 7, 2, 1, 1, 2, 2,
        5, 3, 1, 4, 5, 7, 4, 2, 3, 4, 7, 3, 7, 6, 2, 3, 7, 7, 2, 1, 7, 0, 0, 5,
        1, 7, 3, 0, 4, 7, 5, 4, 7, 5, 0, 5, 6, 6, 4, 0, 7, 1, 0, 3, 5, 2, 0, 5,
        6, 3, 3, 2, 1, 7, 0, 6, 1, 7, 5, 0, 6, 2, 7, 0, 2, 1, 0, 7, 1, 4, 3, 4,
        7, 5, 3, 6, 4, 1, 0, 0, 6, 2, 7, 3, 3, 3, 0, 6, 2, 5, 4, 7, 6, 1, 3, 3,
        0, 2, 5, 3, 0, 6, 1, 2, 6, 4, 2, 4, 0, 6, 6, 7])
tensor([  4,  57, 104,  -1, 144,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         57, 102,  -1,  62,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -

In [4]:
train_dataset = EmoSet(data_root=data_root, num_emotion_classes=num_emotion_classes, phase='train')
valid_dataset = EmoSet(data_root=data_root, num_emotion_classes=num_emotion_classes, phase='val')
test_dataset = EmoSet(data_root=data_root, num_emotion_classes=num_emotion_classes, phase='test')

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
class AttributePredictor(nn.Module):
    def __init__(self, num_classes):
        super(AttributePredictor, self).__init__()
        resnet = models.resnet18(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, num_classes)
    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
def train_attribute_model(train_dataset, valid_dataset, num_classes, attribute_name, epochs=1):
    train_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
    model = AttributePredictor(num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_valid_acc = 0.0
    best_model_wts = model.state_dict()
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}")
        model.train()
        total_loss, correct, total = 0, 0, 0
        i=0
        for batch in train_loader:
            i+=1
            images = batch['image']
            labels = batch[f"{attribute_name}_label_idx"]
            valid_indices = labels >= 0
            images = images[valid_indices]
            labels = labels[valid_indices]
            if len(labels) == 0:
                continue
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()
            total += labels.size(0)
            if i % 20 == 0:
                    if(total==0):
                        total=1
                    batch_accuracy = 100 * correct / total
                    print(f"Batch {i}, Loss: {loss.item():.4f}, Accuracy: {batch_accuracy:.2f}%")
        if(total==0):
            total=1
        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Train Loss: {total_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in valid_loader:
                valid_indices = labels >= 0
                images = images[valid_indices]
                labels = labels[valid_indices]
                if len(labels) == 0:
                    continue
                outputs = model(images)
                correct += (outputs.argmax(dim=1) == labels).sum().item()
                total += labels.size(0)
        if(total==0):
            total=1
        valid_acc = 100 * correct / total
        print(f"Validation Accuracy: {valid_acc:.2f}%")
        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            best_model_wts = model.state_dict().copy()
    model.load_state_dict(best_model_wts)
    print(f"Best Validation Accuracy for {attribute_name}: {best_valid_acc:.2f}%")
    torch.save(model.state_dict(), f"{attribute_name}_predictor.pth")
    print(f"Model for {attribute_name} saved!")
    return model

In [ ]:
attribute_classes = {
    'scene': 254,
    'facial_expression': 6,
    'human_action': 264,
    'brightness': 11,
    'colorfulness': 11,
    'object': 409
}
trained_models = {}
for attr, num_classes in attribute_classes.items():
    print(f"Training model for {attr}...")
    trained_models[attr] = train_attribute_model(train_dataset, valid_dataset, num_classes, attr,  epochs=5)

Training model for scene...
Epoch 1
Batch 20, Loss: 5.1681, Accuracy: 19.01%
Batch 40, Loss: 4.4692, Accuracy: 16.09%
Batch 60, Loss: 2.1848, Accuracy: 14.85%
Batch 80, Loss: 4.4012, Accuracy: 16.39%
Batch 100, Loss: 4.1697, Accuracy: 15.94%
Batch 120, Loss: 4.3619, Accuracy: 15.75%
Batch 140, Loss: 4.2890, Accuracy: 15.26%
Batch 160, Loss: 6.9186, Accuracy: 14.97%
Batch 180, Loss: 3.7724, Accuracy: 14.83%
Batch 200, Loss: 3.7108, Accuracy: 15.03%
Batch 220, Loss: 5.6682, Accuracy: 14.41%
Batch 240, Loss: 4.8617, Accuracy: 14.14%
Batch 260, Loss: 4.4241, Accuracy: 14.20%
Batch 280, Loss: 4.8973, Accuracy: 14.85%
Batch 300, Loss: 5.8158, Accuracy: 14.98%
Batch 320, Loss: 3.0999, Accuracy: 14.94%
Batch 340, Loss: 4.5383, Accuracy: 15.17%
Batch 360, Loss: 4.6202, Accuracy: 15.51%
Batch 380, Loss: 3.3664, Accuracy: 15.20%
Batch 400, Loss: 5.7025, Accuracy: 15.06%
Batch 420, Loss: 5.1143, Accuracy: 14.90%
Batch 440, Loss: 3.7598, Accuracy: 15.26%
Batch 460, Loss: 4.3684, Accuracy: 15.42%
Ba

ZeroDivisionError: division by zero

In [6]:
attribute_classes = {
    'human_action': 264,
    'brightness': 11,
    'colorfulness': 11,
    'object': 409
}
trained_models = {}
for attr, num_classes in attribute_classes.items():
    print(f"Training model for {attr}...")
    trained_models[attr] = train_attribute_model(train_dataset, valid_dataset, num_classes, attr,  epochs=5)

Training model for human_action...


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1
Batch 20, Loss: 6.6359, Accuracy: 10.34%
Batch 40, Loss: 4.9701, Accuracy: 10.45%
Batch 60, Loss: 4.0024, Accuracy: 10.85%
Batch 80, Loss: 5.3674, Accuracy: 13.38%
Batch 100, Loss: 5.0429, Accuracy: 14.38%
Batch 120, Loss: 3.3590, Accuracy: 14.24%
Batch 140, Loss: 4.6282, Accuracy: 13.81%
Batch 160, Loss: 3.2609, Accuracy: 13.35%
Batch 180, Loss: 4.7377, Accuracy: 13.32%
Batch 200, Loss: 3.6289, Accuracy: 13.51%
Batch 220, Loss: 5.8539, Accuracy: 13.33%
Batch 240, Loss: 3.2147, Accuracy: 13.09%
Batch 260, Loss: 5.2700, Accuracy: 12.94%
Batch 280, Loss: 3.1328, Accuracy: 13.33%
Batch 300, Loss: 3.7282, Accuracy: 12.88%
Batch 320, Loss: 3.5053, Accuracy: 13.06%
Batch 340, Loss: 3.9986, Accuracy: 13.10%
Batch 360, Loss: 3.2852, Accuracy: 13.53%
Batch 380, Loss: 3.5696, Accuracy: 13.55%
Batch 400, Loss: 5.8132, Accuracy: 13.98%
Batch 420, Loss: 3.0795, Accuracy: 14.03%
Batch 440, Loss: 4.5378, Accuracy: 14.10%
Batch 460, Loss: 5.1289, Accuracy: 14.18%
Batch 480, Loss: 3.3228, Accur

KeyboardInterrupt: 

In [6]:
attribute_classes = {
    'colorfulness': 11,
    'facial_expression': 6
}
trained_models = {}
for attr, num_classes in attribute_classes.items():
    print(f"Training model for {attr}...")
    trained_models[attr] = train_attribute_model(train_dataset, valid_dataset, num_classes, attr,  epochs=2)

Training model for colorfulness...


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1
Batch 20, Loss: 1.6503, Accuracy: 26.70%
Batch 40, Loss: 1.4592, Accuracy: 30.83%
Batch 60, Loss: 1.4419, Accuracy: 33.16%
Batch 80, Loss: 1.5101, Accuracy: 35.34%
Batch 100, Loss: 1.0453, Accuracy: 37.12%
Batch 120, Loss: 1.1581, Accuracy: 37.94%
Batch 140, Loss: 0.9886, Accuracy: 38.93%
Batch 160, Loss: 1.3310, Accuracy: 40.01%
Batch 180, Loss: 1.1190, Accuracy: 40.13%
Batch 200, Loss: 1.0375, Accuracy: 40.73%
Batch 220, Loss: 1.3488, Accuracy: 41.14%
Batch 240, Loss: 1.1728, Accuracy: 41.93%
Batch 260, Loss: 1.4217, Accuracy: 42.21%
Batch 280, Loss: 0.9751, Accuracy: 42.51%
Batch 300, Loss: 1.1574, Accuracy: 42.85%
Batch 320, Loss: 1.6871, Accuracy: 42.93%
Batch 340, Loss: 1.2241, Accuracy: 43.10%
Batch 360, Loss: 1.0774, Accuracy: 43.09%
Batch 380, Loss: 1.2113, Accuracy: 43.21%
Batch 400, Loss: 1.2370, Accuracy: 43.59%
Batch 420, Loss: 0.9570, Accuracy: 44.11%
Batch 440, Loss: 1.3851, Accuracy: 44.36%
Batch 460, Loss: 1.5444, Accuracy: 44.40%
Batch 480, Loss: 1.4793, Accur

In [8]:
model1=trained_models['colorfulness']
torch.save(model1.state_dict(), f"colorfulness_predictor.pth")

In [9]:
model1=trained_models['facial_expression']
torch.save(model1.state_dict(), f"facial_expression_predictor.pth")

In [6]:
attribute_classes = {
    'scene': 254,
    'facial_expression': 6,
    'human_action': 264,
    'brightness': 11,
    'colorfulness': 11,
    'object': 409
}